The idea is to order the icesat-2 granules separately and download it after a while (few hours). That way we do not have to wait for order to be finished on the NSIDC Server

Obervation: Usually an extra order is returned by the script that is not actually data  
            Perhaps this was the known error on part of NSIDC  

# The following file need to be downloaded


In [1]:
import pandas as pd
import os
import requests
from requests.auth import HTTPBasicAuth

import zipfile
import io

In [2]:
# Other custom modules specific to icesat-2
from icesat2_download import  move_files_from_order, read_atl06, read_atl08, get_api_key

In [3]:
#Create session to store cookie and pass credentials to capabilities url
# email = '' #no required here because we are just downloading the data 
try:
    #from icesat2_search_and_download_ATL import get_api_key
    uid, pswd, token = get_api_key()
    print("Success getting authentication")
except:
    # Input explicitly here
    uid = ''#'Replace with your username'
    pswd = ''#'Repalce with your password'
    #pswd = getpass.getpass('Earthdata Login password: ') #B2f


Success getting authentication


In [4]:
# base_icesat2_path = f'D:/icesat2/{dem_type}/{region}_2' #temp testing only
''' Download the previously ordered icesat2 data and extract zip files
    The order is read from the order.csv file saved earlier in the same folder as order
'''
# Define the path for download path
base_icesat2_path = 'D:/icesat2/rema/region_03_peninsula_south_3' #change based on region
# Read the csv file with order numbers
df = pd.read_csv(f'{base_icesat2_path}/orders.csv', index_col=0)
for idx, ser in df.iterrows():
    #print(idx, ser.values)
    orderIDs = list(ser.values)
    strip = idx
    icesat2_path = base_icesat2_path + '/' + strip
    print(icesat2_path, os.path.exists(icesat2_path))
    for orderID in orderIDs:
        try:
            print(orderID)
            downloadURL = f'https://n5eil02u.ecs.nsidc.org/esir/{orderID}.zip'
            # Create session to store cookie and pass credentials to capabilities url
            # session = requests.session() # being deprecated but still used by nsidc
            session = requests.sessions.Session() #newer
            s = session.get(downloadURL) #Required But still 401 Error
            zip_response = session.get(s.url, auth=(uid,pswd))
            # zip_response = session.get(downloadURL, auth=(uid,pswd)) #But this does not work
            zip_response.raise_for_status()

            #icesat2_path = 'D:/temp'
            with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
                z.extractall(f'{icesat2_path}/downloads')
            # Extract files from downloads folder and remove the downloads folder
            move_files_from_order(icesat2_path)
            print('end:', orderID)
        except:
            print(f'Missing {orderID}')

In [5]:
# turn warnings off
# import warnings

In [6]:
# Parse the downloaded HDF Files (ATL06 or ATL08)
read_atl08(icesat2_path)